In [4]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

import pickle

from random import sample, seed
from tqdm import tqdm
from utils.propagators.directed import PropagateDirected
from utils.metrics import accuracy

seed(42)

In [2]:
# Load Graph.
G = nx.read_gexf("../../state_files/PyPi Network V4.gexf")
# Keep only giant component.
gc_nodes = sorted(nx.connected_components(G.to_undirected()), key = lambda x: len(x), reverse=True)[0]
not_gc_nodes = set(G.nodes()) - gc_nodes
G.remove_nodes_from(not_gc_nodes)

# Load labels.
with open("../../state_files/PyPi Dataframe V4.pickle", "rb") as f:
    df = pickle.load(f)
labels = df["Topic"].dropna().to_dict()
# Remove 'Other' topic.
for n, l in labels.items():
    if 'Other/Nonlisted Topic' in l:
        l.remove('Other/Nonlisted Topic')
# Filter nodes with no topics.
labels = dict(filter(lambda x: len(x[1]) > 0, labels.items()))
# Keep nodes from giant component.
labels = dict(filter(lambda x: x[0] in gc_nodes, labels.items()))

# Split train and test set.
test_size = 0.2

test_nodes = sample(list(labels.keys()), int(test_size*len(labels.keys())))
train_nodes = list(filter(lambda x: x not in test_nodes, labels.keys()))

train_labels = dict(filter(lambda x: x[0] in train_nodes, labels.items()))
test_labels = dict(filter(lambda x: x[0] in test_nodes, labels.items()))

In [ ]:
pl = PropagateDirected(G, train_labels)
final_labels = pl.propagate_all()
acc = accuracy(test_labels, final_labels)